# Semianr 4 - Combinatoris, Graph, Boolean Logic

## SQL + Python

**SQL**: Structured Query Language

In [ ]:
# !pip install --upgrade pip

In [ ]:
# !pip install mysql

In [ ]:
# !pip install mysql-connector-python

In [ ]:
# LOCalhost/phpmyadmin/index.php

In [2]:
import mysql.connector as sql
import pandas as pd

In [7]:
conn = sql.connect(
    host='localhost',
    user='root',
    password='',
    database='Allocation'
    )

In [8]:
cursor = conn.cursor()

cursor.execute("""
    SELECT * FROM projects
""")

cursor.fetchall()

[(1, 'a', '10.05.2020', '15.05.2020'),
 (2, 'b', '13.05.2020', '27.07.2020'),
 (3, 'c', '08.07.2020', '30.07.2020'),
 (4, 'd', '11.12.2020', '29.12.2020'),
 (5, 'e', '06.11.2020', '07.11.2020')]

In [9]:
querry_1 = 'SELECT * FROM projects'

df_projects = pd.read_sql(querry_1, conn)
df_projects

/var/folders/25/j6_kp1v55376c5ldy5f8ns1r0000gn/T/ipykernel_1144/636419232.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_projects = pd.read_sql(querry_1, conn)


,ID,name,start_time,end_time
0,1,a,10.05.2020,15.05.2020
1,2,b,13.05.2020,27.07.2020
2,3,c,08.07.2020,30.07.2020
3,4,d,11.12.2020,29.12.2020
4,5,e,06.11.2020,07.11.2020


In [10]:
querry_2 = 'SELECT * FROM resources'

df_resources = pd.read_sql(querry_2, conn)
df_resources

/var/folders/25/j6_kp1v55376c5ldy5f8ns1r0000gn/T/ipykernel_1144/998539140.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resources = pd.read_sql(querry_2, conn)


,ID,name
0,1,A
1,2,B
2,3,C
3,4,D
4,5,E


In [11]:
querry_3 = 'SELECT * FROM allocation'

df_allocation = pd.read_sql(querry_3, conn)
df_allocation

/var/folders/25/j6_kp1v55376c5ldy5f8ns1r0000gn/T/ipykernel_1144/2963314014.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_allocation = pd.read_sql(querry_3, conn)


,ID,ID_RES,ID_PROJ
0,1,1,1
1,2,5,3
2,3,2,4
3,4,4,5
4,5,3,2
5,6,2,1
6,7,3,1
7,8,2,2


## All feasible solutions

In [13]:
feasible_solutions = {}

for project_index, project_rows in df_projects.iterrows():
    project_id = project_rows['ID']
    res_ids = df_allocation[df_allocation['ID_PROJ'] == project_id]['ID_RES']
    feasible_solutions[project_id] = []
    
    for res_id in res_ids:
        feasible_solutions[project_id].append(res_id)
        
feasible_solutions

{1: [1, 2, 3], 2: [3, 2], 3: [5], 4: [2], 5: [4]}

In [14]:
def date_to_sec(date):
    return time.mktime(datetime.datetime.strptime(date, "%d.%m.%Y").timetuple())

In [18]:
import time
import datetime

# keys = Project IDs df_expertise
keys = list(feasible_solutions.keys())

def is_valid_solution(solution):
    
    for i in range(len(keys)-1):
        for j in range(i+1, len(keys)):
                
            if solution[i] == solution[j]:
                start_i = date_to_sec(df_projects["start_time"][i])
                end_i = date_to_sec(df_projects["end_time"][i])
                
                start_j = date_to_sec(df_projects["start_time"][j])
                end_j = date_to_sec(df_projects["end_time"][j])
                
                if start_i <= start_j:
                    if end_i >= start_j:
                        return False
                    
                elif start_i >= start_j:
                    if end_j >= start_i:
                        return False
    
    return True

# Checking a single solution
# print(is_valid_solution([1, 2, 5, 3, 4]))

# print(is_valid_solution([1, 2, 2, 3, 4]))

In [19]:
answer = None

def recursive(solution=[], depth=0):
    global answer
    if depth>=len(keys):
        if(is_valid_solution(solution)):
            answer = solution
        return
    
    values = feasible_solutions[keys[depth]]
    
    for v in values:
        if answer:
            return answer
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

['1', '3', '5', '2', '4']

In [20]:
def recursive(solution=[], depth=0):
    if (depth>=len(keys)):
        if(is_valid_solution(solution)):
            print(', '.join(solution))
        return
    
    values = feasible_solutions[keys[depth]]
    
    for v in values:
        next_solution = solution.copy()
        next_solution.append(str(v))
        recursive(next_solution, depth+1)
        
recursive()

1, 3, 5, 2, 4
1, 2, 5, 2, 4
2, 3, 5, 2, 4
3, 2, 5, 2, 4


## How to create tables from Python

In [21]:
cursor.execute("""
    CREATE TABLE employeeinfo(
    ID INT AUTO_INCREMENT PRIMARY KEY,
    name varchar(50) NOT NULL,
    position varchar(50) NOT NULL
    )
    """)

In [22]:
cursor.execute("""
    SHOW TABLES
    """)

cursor.fetchall()

[('allocation',), ('employeeinfo',), ('projects',), ('resources',)]

In [23]:
querry = """
    INSERT INTO employeeinfo (name, position) VALUES (%s, %s)
"""

value = ('Slava', 'Mathematition')
cursor.execute(querry, value)

In [26]:
values = [('Tom', 'Programmer'),
         ('Artem', 'Programmer'),
         ('Oleg', 'Data Scientist'),
         ('Anna', 'Data Scientist')]
cursor.executemany(querry, values)

In [27]:
conn.commit()

In [28]:
# How to return uniqu data from my table?

cursor.execute('SELECT DISTINCT position FROM employeeinfo')
cursor.fetchall()

[('Mathematition',), ('Programmer',), ('Data Scientist',)]

## How to check if a Binary Tree is a Binary Search Tree (BST) or not??!!

To solve the problem we should know **What are the characteristics of the BST?**

- The left subtree of a node contains only nodes with keys less than the node’s key.
- The right subtree of a node contains only nodes with keys greater than the node’s key.
- Both the left and right subtrees must also be binary search trees.
- Each node (item in the tree) has a distinct key.


**Steps:**

1) If the current node is null then return true

2) If the value of the left child of the node is greater than or equal to the current node then return false

3) If the value of the right child of the node is less than or equal to the current node then return false

4) If the left subtree or the right subtree is not a BST then return false

5) Else return true

In [37]:
# Method 1: Use in-order 

# Method 2:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None
        
def is_binary_tree(tree):
    if tree is None:
        return True
    
    left_is_bin = tree.left is None or \
        tree.left.data < tree.data and is_binary_tree(tree.left)
    
    right_is_bin = tree.right is None or \
        tree.right.data > tree.data and is_binary_tree(tree.right)
    
    return left_is_bin and right_is_bin

tree = Node(5)
tree.left = Node(3)
tree.left.left = Node(1)
tree.left.right = Node(5)
tree.right = Node(12)

if is_binary_tree(tree):
    print('It\'s binary search tree!')
          
else:
    print('It\'s not')

It's binary search tree!


In [38]:
## This program doesn't consider this type of counter example
"""
tree = Node(5)
tree.left = Node(3)
tree.left.left = Node(1)
tree.left.right = Node(6)
tree.right = Node(12)
"""
## Try to fix the provided algorithm

<YOUR CODE>